## Kaggle：观察暗世界大赛

---

#### 介绍

观察暗世界大赛是 Kaggle 官网上发布的一个 [预测大赛](http://www.kaggle.com/c/DarkWorlds) 。而本次大赛的冠军所使用的解决方法正是贝叶斯推断。因此，本次实验将对冠军的解决方法、核心思路进行讲解和实现。

#### 知识点

- 暗物质的观测
- 先验的确定
- 欧氏距离
- 数据分析
- 数据可视化


---

在进行实验前，我们还是需要加载 PyMC 库，这里请直接运行下面几段代码（具体解释，已在第二个实验中给出）：

In [ ]:
!pip install pymc3==3.8

In [ ]:
#卸载当前环境的 numpy，防止与新版本冲突
!pip  uninstall -y numpy
#更新 numpy
!pip install -U numpy 

In [ ]:
#重启内核
import os
os._exit(00)

In [ ]:
# 测试是否安装成功
import pymc3 as pm
# 输出版本号
print(pm.__version__)

### 观察暗世界大赛

宇宙其实是很大的，而我们能够肉眼观测到的东西却是很少的。这是因为，在宇宙中存在很多既不发光也不吸收光的物质形式，这就是暗物质。

虽然我们无法肉眼观测这种物质，但它不可忽视。如果大规模的暗物质聚集在一起的话，会形成暗物质光晕。虽然这种光晕是暗的，但是它能够扭曲时空。任何接近暗物质的光线的路径都会发生改变，进而发生弯曲，最终使星系变成了椭圆。如下图所示。

<img src="https://doc.shiyanlou.com/courses/1520/1166617/6d9e1c2d2e38bde8e624eea5e55075f3-0" width = 730>

这个比赛要求参赛者预测这些暗物质光晕的可能位置。这个比赛持续了很久时间，最终由  [Tim Salimans](http://timsalimans.com/) 获得了比赛冠军。而他使用就是贝叶斯推断去寻找暗物质光源的可能位置。在获得许可下，我们在这里对他的方案进行阐述和讲解，他的解题思路如下：

1. 构建一个光晕位置 p(x) 的先验分布，即在观察数据之前，我们需要给出光晕的预期位置。
2. 给定暗物质光晕的位置，构造在该光晕位置下的星系椭圆率模型 $p(e|x)$。
3. 使用贝叶斯法得到光晕的后验分布
4. 最小化后验分布的期望损失：$\hat{x} = \arg \min_{\text{prediction} } E_{p(x|e)}[ L( \text{prediction}, x) ]$。

由于 Tim 所提出的损失非常复杂，共有 160 多行，且包含了各种公式，并试图在欧式距离上无偏移地衡量预测进度。因此，我们会直接给出， 你无需仔细阅读，我也不建议你仔细阅读，除非你有非常非常良好的数学功底和动力。

同样，我们还会给出一个画图的 py 文件，用于画出星系的图像。

上面两个接口文件可以通过点击这里得到：[损失文件](https://labfile.oss.aliyuncs.com/courses/1520/DarkWorldsMetric.py)， [画图文件](https://labfile.oss.aliyuncs.com/courses/1520/draw_sky2.py)，下载到本地仔细阅读。

接下来，让我们把这两个 py 文件下载下来：

In [ ]:
!wget -nc "https://labfile.oss.aliyuncs.com/courses/1520/DarkWorldsMetric.py"
!wget -nc "https://labfile.oss.aliyuncs.com/courses/1520/draw_sky2.py"

接下来，我们将利用这两个文件，对 Tim 的优胜方案进行复现。

 #### 数据初始化 

让我们先来下载数据：

In [ ]:
!wget -nc "https://labfile.oss.aliyuncs.com/courses/1520/lab10_data.zip"
!unzip -o lab10_data.zip

文件夹的结构如下:

In [ ]:
!ls Train_Skies/Train_Skies/

该数据集实际上是 300 份单独的文件，每个文件分别代表一个星空。在一个文件（星空）中，有 300~700 个星系。每个星系都有一个与之关联的坐标（x，y），范围从 0 到 4200，和该星系的椭圆参数 $e_1$ 和 $e_2$。让我们随机选择一个星空对其进行可视化。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.pylabtools import figsize
import scipy.stats as stats
from draw_sky2 import draw_sky
%matplotlib inline

n_sky = 3  # 选择第三个星系
# 读取csv文件
data = np.genfromtxt("Train_Skies//Train_Skies/\
Training_Sky%d.csv" % (n_sky),
                     dtype=None,
                     skip_header=1,
                     delimiter=",",
                     usecols=[1, 2, 3, 4])
print("Data on galaxies in sky %d." % n_sky)
print("position_x, position_y, e_1, e_2 ")
print(data[:3])

# 只需将data 传入我们之前下载的工具类中即可
fig = draw_sky(data)
plt.title("Galaxy positions and ellipcities of sky %d." % n_sky)
plt.xlabel("x-position")
plt.ylabel("y-position")

上图为一个星空（文件）的可视化，可以看到里面有很多的星系（椭圆），这些星系的椭圆形状是根据椭圆参数 $e_1,e_2$ 画出来的。

除却这 300 个文件之外，还存在一个文件 `Training_halos.csv`。让我们来加载它：

In [ ]:
import pandas as pd
skys_halos = pd.read_csv("Training_halos.csv")
print("大小：", skys_halos.shape)
skys_halos.head()

从上面的结果中可以看出，`Training_halos.csv` 文件存的是这 300 个星空的训练数据，里面分别有暗物质光晕的个数(numberHalos)，和这些光晕的位置分布 `(halo_x1	halo_y1	halo_x2	halo_y2	halo_x3	halo_y3)`。例如 (1，1，0，0，0，0)  表示该星系只存在一个光晕，位置在（1,1）。即横纵坐标都为 0 表示没有光晕。

#### 先验的确定

从上面的文件中，我们也可以很明显的看到每个星空有 1~3 个暗物质光晕（因为只有三对暗物质晕的坐标）。

因此 Tim 定义了光晕位置的先验是一个均匀分布，如下：

\begin{align}
& x_i \sim \text{Uniform}( 0, 4200)\\\\
& y_i \sim \text{Uniform}( 0, 4200), \;\; i=1,2,3\\\\
\end{align}

Tim 和其他参赛者注意到大多数星空含有一个大的光晕，如果还有其他光晕的话，那么这些的一般都很小。这可能是因为大光晕的质量非常大，更能够影响周围的星系。

他认为，大光晕的质量服从一个 40~180 之间的对数的均匀分布。即：

$$  m_{\text{large} } = \log \text{Uniform}( 40, 180 ) $$


上列式子，在 PyMC3 中的表示如下： 

    exp_mass_large = pm.Uniform("exp_mass_large", 40, 180)
    mass_large = pm.Deterministic("mass_large", np.log(exp_max_large))

这就是，为什么我们叫它为对数的均匀分布的原因。而对于小的光晕，Tim 直接把它们的质量设定为 20 的对数。由于小的星系影响较小，且为了加快算法的收敛效率，所以 Tim 把它们定一个常数。

然后，假设每个星系的椭圆率和光晕的位置，星系和光晕之间的距离以及光晕的质量有关。并且，我们从相关文献中找到了一个较为合理的关系：

$$ e_i | ( \mathbf{x}, \mathbf{y} ) \sim \text{Normal}( \sum_{j = \text{halo positions} }d_{i,j} m_j f( r_{i,j} ), \sigma^2 ) $$

- $e_i$ 表示第 i 个星系的椭圆率的矢量
- （x，y）表示光晕位置
- $d_{i,j}$ 表示正切方向（星系 i 的光线受光晕 j 的影响所弯曲的方向）
- $m_j$ 是光晕 j 的质量



$f(r_{i,j})$ 是光晕 $j$ 和星系 $i$ 的欧式距离的倒数。简单的来说，对于大的光晕，函数 $f$ 的定义如下：

$$ f( r_{i,j} ) = \frac{1}{\min( r_{i,j}, 240 ) } $$


对于小的光晕，函数 $f$ 的定义如下：

$$ f( r_{i,j} ) = \frac{1}{\min( r_{i,j}, 70 ) } $$


好的，接下来，让我们先来实现这些函数：

In [ ]:
def euclidean_distance(x, y):
    # 计算 d
    return np.sqrt(((x - y)**2)).sum(axis=1)


def f_distance(gxy_pos, halo_pos, c):
    # 计算 f
    return T.maximum(euclidean_distance(gxy_pos, halo_pos), c)[:, None]


def tangential_distance(glxy_position, halo_position):
    # 考虑正切方向下的距离
    delta = glxy_position - halo_position
    t = (2*T.arctan(delta[:, 1]/delta[:, 0]))
    return T.stack([-T.cos(t), -T.sin(t)], axis=1)

#### 模型的训练

接下来，让我们根据上面的公式，定义模型(需要运行 3~5 min)：

In [ ]:
import theano.tensor as T
with pm.Model() as model:
    # 设置大的光晕的质量分布
    mass_large = pm.Uniform("mass_large", 40, 180)

    # 设置光晕的位置
    halo_position = pm.Uniform("halo_position", 0, 4200, shape=(1, 2))
    # 计算上述公式中的μ的值，也就是正态分布的第一个参数，
    mean = pm.Deterministic("mean", mass_large /
                            f_distance(T.as_tensor(data[:, :2]), halo_position, 240) *
                            tangential_distance(T.as_tensor(data[:, :2]), halo_position))

    # 根据正态分布得到椭圆率矢量，并且与真实的椭圆率矢量进行固定。即把真实数据纳入模型中
    ellpty = pm.Normal("ellipcity", mu=mean, tau=1./0.05, observed=data[:, 2:])
model

接下来，让我们对装载好的模型进行训练（由于模型复杂，模型加载需要一点时间，下面代码段大致运行 3~5 min，请耐心等待）：

注：下面代码可能会有两个进度条产生，请耐心等待：

In [ ]:
with model:
    # 设置需要在意的变量
    # 需要1使用自动微分变分来进行变量推理
    trace = pm.sample(1000, init='advi', n_init=10000)
print("训练完成")

上面代码在输出“训练完成”字样后，才算完成，请耐心等待。

为了方便观察与分析，接下来让我们画出光晕位置的后验分布（由于分布是一个个数据点集合，因此，我们可以直接将其画在图中）：

In [ ]:
# 所有的预测可能，一共2000个坐标样本
t = trace["halo_position"].reshape(2000, 2)

fig = draw_sky(data)
plt.title("Galaxy positions and ellipcities of sky %d." % n_sky)
plt.xlabel("x-position")
plt.ylabel("y-position")
plt.scatter(t[:, 0], t[:, 1], alpha=0.015, c="r")
plt.xlim(0, 4200)
plt.ylim(0, 4200)

如上图，红色区域是由很多很多的数据点所组成的，这些数据点就是我们对该星系光晕位置的预测。接下来，让我们加载并从 `Training_halos.csv` 中获得真实光晕的位置：

In [ ]:
halo_data = np.genfromtxt("Training_halos.csv",
                          delimiter=",",
                          usecols=[1, 2, 3, 4, 5, 6, 7, 8, 9],
                          skip_header=1)
print(halo_data[n_sky-1])

其中第三列和第四列的光晕代表的就是 x，y 坐标。接下来，让我们把这个真实坐标也画到星空中：

In [ ]:
fig = draw_sky(data)
plt.title("Galaxy positions and ellipcities of sky %d." % n_sky)
plt.xlabel("x-position")
plt.ylabel("y-position")
plt.scatter(t[:, 0], t[:, 1], alpha=0.015, c="r")
plt.scatter(halo_data[n_sky-1][3], halo_data[n_sky-1][4],
            label="True halo position",
            c="k", s=70)
plt.legend(scatterpoints=1, loc="lower left")
plt.xlim(0, 4200)
plt.ylim(0, 4200)

print("真实的光晕位置:", halo_data[n_sky][3], halo_data[n_sky][4])

从上图可以很清楚的看到，暗物质光晕的真实位置（黑色圆点）和预测的位置大致相似。

接下来让我们取分布中，所有值的平均值，作为最后的暗物质光晕的预测位置。

In [ ]:
mean_posterior = t.mean(axis=0).reshape(1, 2)
print("最终预测位置为：", mean_posterior)

最后，我们把我们的最终预测位置（用平均值预测出来的最终位置）和真实位置放入损失函数，计算出损失：

In [ ]:
# 导入损失函数
from DarkWorldsMetric import main_score

_halo_data = halo_data[n_sky-1]
# 真实坐标
nhalo_all = _halo_data[0].reshape(1, 1)
x_true_all = _halo_data[3].reshape(1, 1)
y_true_all = _halo_data[4].reshape(1, 1)
x_ref_all = _halo_data[1].reshape(1, 1)
y_ref_all = _halo_data[2].reshape(1, 1)

# 预测坐标
sky_prediction = mean_posterior

print("使用平均值作为最终结果得到的损失:")
score1 = main_score(nhalo_all, x_true_all, y_true_all,
                    x_ref_all, y_ref_all, sky_prediction)

好的，至此，我们完成了一个星系的暗物质光晕的预测，而其他小的光晕位置的预测雷同（只需修改几个参数即可）。由于计算机资源短缺和运行需要大量时间的问题，这里我们不对其他星云以及其他光晕进行预测。如果你有兴趣的话，可以将上面代码打包成相应函数，然后遍历所有星云，得到所有的光晕的位置，然后计算平均损失。

### 实验总结

首先，我们对每个光晕位置设置了一个均匀分布的先验，然后根据文献找到了椭圆率和光晕位置、距离、质量的关系。然后把真实数据的椭圆率与模型结合，最后通过贝叶斯推断计算出来所有参数的后验分布，即光晕的真实估计位置。并且将该位置和真实数据集中的位置进行比较，发现差距不大，很好的证明了模型的准确性。

<hr><div style="color: #999; font-size: 12px;"><i class="fa fa-copyright" aria-hidden="true"> 本课程内容版权归实验楼所有，禁止转载、下载及非法传播。</i></div>